In [90]:
import numpy as np
import os
from torch import nn
import torch
from torch.autograd import Variable

In [91]:
FILE_DIR = 'eeg_fpz_cz/'
data = []
for each in os.listdir(FILE_DIR):
    this_data = np.load(FILE_DIR+each, allow_pickle=True)
    this_x = this_data['x']
    this_y = this_data['y']
    data.append((this_x, this_y))
print(len(data))

#[(x, y), ……]

39


In [92]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
idx = np.arange(39)
np.random.shuffle(idx)
split = 30
train_idx, test_idx = idx[:split], idx[split:]


In [93]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM( 
            input_size=1,     
            hidden_size=64,    
            num_layers=1,   
            batch_first=True,   
        )
        self.out = nn.Linear(64, 5) 
    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)
        out = self.out(r_out[:, -1, :])
        return out
rnn = RNN()

In [94]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.003)
loss_func = nn.CrossEntropyLoss() 

In [101]:
for epoch in range(1000):
    print(epoch)
    for i in range(len(train_idx)):
        trainx = data[i][0]
        trainy = data[i][1]
        b_x = Variable(torch.from_numpy(trainx))   
        b_y = Variable(torch.from_numpy(trainy))  
        b_y = b_y.type(torch.LongTensor)

        output = rnn(b_x)              
        loss = loss_func(output, b_y) 
        optimizer.zero_grad()      
        loss.backward()              
        optimizer.step()    
        print(loss.item())

0
1.5987303256988525
1.5928114652633667


KeyboardInterrupt: 

In [102]:
#test
output = rnn(torch.from_numpy(data[test_idx[0]][0]))

In [110]:
output[0]

tensor([-0.0267,  0.0686,  0.1568, -0.0165, -0.1099], grad_fn=<SelectBackward>)

In [109]:
data[test_idx[0]][1][300]

4